## Spark 설정하기
1. SparkConf, SparkContext, pandas 임포트
2. SparkConf 객체 생성 및 설정
3. SparkContext 객체 생성

In [5]:
# import
import findspark
findspark.init()

from pyspark import SparkConf, SparkContext
import pandas as pd

In [6]:
conf = SparkConf().setMaster("local").setAppName("key-value-rdd")
sc = SparkContext(conf=conf)

In [7]:
sc

<SparkContext master=local appName=key-value-rdd>

## Key-Value RDD 연산 예제
1. Python List로부터 RDD 객체 생성: SparkContext.parallelize() 
2. Single Value RDD에서 Key-Value RDD로 변환: RDD.map() 
3. Key-Value RDD 값 세기: RDD.reduceByKey() 

+ collect() : RDD 모든 값을 출력
+ reduceByKey() : Key를 기준으로 

In [8]:
# Python list로부터 RDD 객체 생성
rdd = sc.parallelize([
    '2023-03-28', '2023-03-28', '2023-03-28',
    '2023-03-29',
    '2023-03-30', '2023-03-30'
])

In [9]:
# RDD 값을 Key-Value 형태로 변환
pairs = rdd.map(lambda x: (x, 1))

In [10]:
# collect(): RDD 모든 값을 출력 (신중히 사용할 것)
pairs.collect()

[('2023-03-28', 1),
 ('2023-03-28', 1),
 ('2023-03-28', 1),
 ('2023-03-29', 1),
 ('2023-03-30', 1),
 ('2023-03-30', 1)]

In [11]:
# reduceByKey(): Key 값을 기준으로 집계
reduced = pairs.reduceByKey(lambda a, b: a + b)

In [12]:
# 집계 결과 출력
reduced.collect()

[('2023-03-28', 3), ('2023-03-29', 1), ('2023-03-30', 2)]

## Restaurant 예제
식당 유형 별 평균 리뷰 수를 계산
1. csv 파일로부터 RDD 생성: SparkContext.textFile() 
2. RDD 헤더 부분 변수에 저장: RDD.first() 
3. 헤더만 제거된 RDD로 변환: RDD.filter(lambda ...)
4. category와 reviews 값을 추출하는 함수 정의
5. category, reviews로 구성된 RDD로 변환: RDD.map()
6. Key: category, Value: (reviews, 1)로 구성된 Key-Value RDD로 변환: RDD.mapValues()
7. Key 를 기준으로 reduction: RDD.reduceByKey()
8. Key 별 평균 reviews 계산: RDD.mapValues()

In [8]:
# 파일 불러오기
filename = "restaurant_reviews.csv"
path = './data/'
lines = sc.textFile(path + filename)

In [9]:
# RDD 값 출력
lines.collect()

['id,item,cateogry,reviews',
 '0,짜장면,중식,125',
 '1,짬뽕,중식,235',
 '2,김밥,분식,32',
 '3,떡볶이,분식,534',
 '4,라멘,일식,223',
 '5,돈가스,일식,52',
 '6,우동,일식,12',
 '7,쌀국수,아시안,312',
 '8,햄버거,패스트푸드,12',
 '9,치킨,패스트푸드,23']

In [10]:
# 헤더 추출
header = lines.first()

In [11]:
# 헤더 제거
filtered_lines = lines.filter(lambda row: row != header)

In [15]:
# 헤더가 제거된 RDD 출력
filtered_lines.collect()

['0,짜장면,중식,125',
 '1,짬뽕,중식,235',
 '2,김밥,분식,32',
 '3,떡볶이,분식,534',
 '4,라멘,일식,223',
 '5,돈가스,일식,52',
 '6,우동,일식,12',
 '7,쌀국수,아시안,312',
 '8,햄버거,패스트푸드,12',
 '9,치킨,패스트푸드,23']

In [19]:
# 카테고리와 리뷰수 추출하는 함수

def parse(row):
    # '0,짜장면,중식,125'
    fields = row.split(',')
    category = fields[2]
    reviews = int(fields[3])
    
    return (category, reviews)

In [20]:
# parse 함수를 이용한 map 연산
category_reviews = filtered_lines.map(parse)

In [21]:
category_reviews.collect()

[('중식', 125),
 ('중식', 235),
 ('분식', 32),
 ('분식', 534),
 ('일식', 223),
 ('일식', 52),
 ('일식', 12),
 ('아시안', 312),
 ('패스트푸드', 12),
 ('패스트푸드', 23)]

+ [Q] reviews = int(fields[3]) 를 reviews = fields[3] 으로 바꾸면 어떻게 될까?

In [22]:
# Key: category, Value: (reviews, 1)로 구성된 Key-Value RDD로 변환
category_reviews_count = category_reviews.mapValues(lambda x: (x, 1))


In [23]:
category_reviews_count.collect()

[('중식', (125, 1)),
 ('중식', (235, 1)),
 ('분식', (32, 1)),
 ('분식', (534, 1)),
 ('일식', (223, 1)),
 ('일식', (52, 1)),
 ('일식', (12, 1)),
 ('아시안', (312, 1)),
 ('패스트푸드', (12, 1)),
 ('패스트푸드', (23, 1))]

In [25]:
# 식당 유형별 리뷰 수, 식당 수 집계
reduced = category_reviews_count.reduceByKey(lambda x, y: (x[0] + y[0], x[1] + y[1]))

In [26]:
reduced.collect()

[('중식', (360, 2)),
 ('분식', (566, 2)),
 ('일식', (287, 3)),
 ('아시안', (312, 1)),
 ('패스트푸드', (35, 2))]

In [27]:
# 식당 유형별 평균 리뷰 수 = 리뷰 수 / 식당 수
averages = reduced.mapValues(lambda x: x[0] / x[1])

In [28]:
averages.collect()

[('중식', 180.0),
 ('분식', 283.0),
 ('일식', 95.66666666666667),
 ('아시안', 312.0),
 ('패스트푸드', 17.5)]

In [ ]:
# parse() 함수 없이 바로 lambda 함수로 구현하기
paired = filtered_lines.map(lambda x: (x.split(',')[2], (int(x.split(',')[3]), 1)))

In [ ]:
paired.collect()

In [ ]:
reduced = paired.reduceByKey(lambda x, y: (x[0] + y[0], x[1] + y[1]))

In [ ]:
reduced.collect()